<a href="https://colab.research.google.com/github/quanghuynguyen3007/QuantiumDataAnalyst/blob/main/Quantium_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantium virtual internship - Retail strategy and analytics

This file is a solution of the Quantium Internship task 1.

In [1]:
## Load required libraries and datasets
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import re

customerData = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Quantium/Task1/QVI_purchase_behaviour.csv')
transactionData = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Quantium/Task1/QVI_transaction_data.xlsx')

## Exploratory data analysis
The first step in any analysis is to first understand the data. Let's take a look
at each of the datasets provided.

In [2]:
#Examining transaction data
transactionData.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8


Let's check if columns we would expect to be numeric are in numeric form and date.

In [3]:
transactionData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264836 entries, 0 to 264835
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DATE            264836 non-null  int64  
 1   STORE_NBR       264836 non-null  int64  
 2   LYLTY_CARD_NBR  264836 non-null  int64  
 3   TXN_ID          264836 non-null  int64  
 4   PROD_NBR        264836 non-null  int64  
 5   PROD_NAME       264836 non-null  object 
 6   PROD_QTY        264836 non-null  int64  
 7   TOT_SALES       264836 non-null  float64
dtypes: float64(1), int64(6), object(1)
memory usage: 16.2+ MB


We can see that the date column is in an integer format. Let's change this to a
date format.

In [4]:
#### Convert DATE column to a date format
transactionData['DATE'] = pd.to_datetime(transactionData['DATE'], origin = pd.Timestamp('1899-12-30'),unit='D')
transactionData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264836 entries, 0 to 264835
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   DATE            264836 non-null  datetime64[ns]
 1   STORE_NBR       264836 non-null  int64         
 2   LYLTY_CARD_NBR  264836 non-null  int64         
 3   TXN_ID          264836 non-null  int64         
 4   PROD_NBR        264836 non-null  int64         
 5   PROD_NAME       264836 non-null  object        
 6   PROD_QTY        264836 non-null  int64         
 7   TOT_SALES       264836 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(5), object(1)
memory usage: 16.2+ MB


We should check that we are looking at the right products by examining PROD_NAME. Looks like we are definitely looking at potato chips but how can we check that
these are all chips? We can do some basic text analysis by summarising the
individual words in the product name.

In [5]:
# Examine the words in PROD_NAME to see if there are any incorrect entries
# such as products that are not chips
transactionData['PROD_NAME'].unique()[:10,]

array(['Natural Chip        Compny SeaSalt175g',
       'CCs Nacho Cheese    175g',
       'Smiths Crinkle Cut  Chips Chicken 170g',
       'Smiths Chip Thinly  S/Cream&Onion 175g',
       'Kettle Tortilla ChpsHny&Jlpno Chili 150g',
       'Old El Paso Salsa   Dip Tomato Mild 300g',
       'Smiths Crinkle Chips Salt & Vinegar 330g',
       'Grain Waves         Sweet Chilli 210g',
       'Doritos Corn Chip Mexican Jalapeno 150g',
       'Grain Waves Sour    Cream&Chives 210G'], dtype=object)

As we are only interested in words that will tell us if the product is chips or
not, let's remove all words with digits and special characters such as '&' from our
set of product words. Then look at the most common words by counting the number of times a word appears and sorting them by this frequency in order of highest to lowest frequency

In [6]:
from collections import Counter

# Create text cleaning function for PROD_NAME feature
def clean_text(text):
    text = re.sub('[&/]', ' ', text)
    text = re.sub('\d\w*', ' ', text) 
    return str(text)

# Apply text cleaning function to PROD_NAME column
ar = transactionData['PROD_NAME'].apply(clean_text).unique()
# Create bag of works
bag_of_word = [word for x in ar for word in x.split()]
pd.DataFrame.from_dict(Counter(bag_of_word), orient='index', columns=["frequency"]).sort_values('frequency', ascending=False).head(20)

,frequency
Chips,21
Smiths,16
Crinkle,14
Cut,14
Kettle,13
Salt,12
Cheese,12
Original,10
Doritos,9
Chip,9


There are salsa products in the dataset but we are only interested in the chips
category, so let's remove these.

In [7]:
#### Remove salsa products
transactionData = transactionData.loc[~transactionData['PROD_NAME'].str.lower().str.contains('salsa')]

Next, we check summary statistics such as mean, min and max
values or null for each feature to see if there are any obvious outliers in the data.

In [8]:
display(transactionData.describe())
print(transactionData.isna().any())

,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_QTY,TOT_SALES
count,246742.000000,2.467420e+05,2.467420e+05,246742.000000,246742.000000,246742.000000
mean,135.051098,1.355310e+05,1.351311e+05,56.351789,1.908062,7.321322
std,76.787096,8.071528e+04,7.814772e+04,33.695428,0.659831,3.077828
min,1.000000,1.000000e+03,1.000000e+00,1.000000,1.000000,1.700000
25%,70.000000,7.001500e+04,6.756925e+04,26.000000,2.000000,5.800000
50%,130.000000,1.303670e+05,1.351830e+05,53.000000,2.000000,7.400000
75%,203.000000,2.030840e+05,2.026538e+05,87.000000,2.000000,8.800000
max,272.000000,2.373711e+06,2.415841e+06,114.000000,200.000000,650.000000


DATE              False
STORE_NBR         False
LYLTY_CARD_NBR    False
TXN_ID            False
PROD_NBR          False
PROD_NAME         False
PROD_QTY          False
TOT_SALES         False
dtype: bool


There are no nulls in the columns but product quantity appears to have an outlier which we should investigate further. Let's investigate further the case where 200 packets of chips are bought in one transaction.

In [9]:
#### Filter the dataset to find the outlier
transactionData.loc[transactionData['PROD_QTY'] == 200]

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
69762,2018-08-19,226,226000,226201,4,Dorito Corn Chp Supreme 380g,200,650.0
69763,2019-05-20,226,226000,226210,4,Dorito Corn Chp Supreme 380g,200,650.0


There are two transactions where 200 packets of chips are bought in one transaction
and both of these transactions were by the same customer.

In [10]:
#### Let's see if the customer has had other transactions
transactionData['LYLTY_CARD_NBR'] == 226000

0         False
1         False
2         False
3         False
4         False
          ...  
264831    False
264832    False
264833    False
264834    False
264835    False
Name: LYLTY_CARD_NBR, Length: 246742, dtype: bool

It looks like this customer has only had the two transactions over the year and is not an ordinary retail customer. The customer might be buying chips for commercial purposes instead. We'll remove this loyalty card number from further analysis.

In [11]:
#### Filter out the customer based on the loyalty card number
transactionData = transactionData.loc[transactionData['LYLTY_CARD_NBR'] != 226000]
#### Re-examine transaction data
transactionData.describe()

,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_QTY,TOT_SALES
count,246740.000000,2.467400e+05,2.467400e+05,246740.000000,246740.000000,246740.000000
mean,135.050361,1.355303e+05,1.351304e+05,56.352213,1.906456,7.316113
std,76.786971,8.071520e+04,7.814760e+04,33.695235,0.342499,2.474897
min,1.000000,1.000000e+03,1.000000e+00,1.000000,1.000000,1.700000
25%,70.000000,7.001500e+04,6.756875e+04,26.000000,2.000000,5.800000
50%,130.000000,1.303670e+05,1.351815e+05,53.000000,2.000000,7.400000
75%,203.000000,2.030832e+05,2.026522e+05,87.000000,2.000000,8.800000
max,272.000000,2.373711e+06,2.415841e+06,114.000000,5.000000,29.500000


That's better. Now, let's look at the number of transaction lines over time to see
if there are any obvious data issues such as missing data.


In [12]:
#### Count the number of transactions by date
transactionData.groupby("DATE").count().shape[0]

364

There's only 364 rows, meaning only 364 dates which indicates a missing date. Let's
create a sequence of dates from 1 Jul 2018 to 30 Jun 2019 and use this to create a
chart of number of transactions over time to find the missing date.


In [26]:
transactions_by_date = transactionData.groupby(['DATE'])['TOT_SALES'].count()
idx = pd.date_range('2018-07-01', '2019-06-30')
transactions_by_date = transactions_by_date.reindex(idx, fill_value=0)
fig = px.line(transactions_by_date,
              title="Total sales by date",
              labels={'value': 'Date', 'index':'Total Sales'},
              template='plotly_dark')
fig.show()

We can see that there is an increase in purchases in December and a break in late
December. Let's zoom in on this.

In [25]:
#### Filter to December and look at individual days
transactions_by_date = transactionData.groupby(['DATE'])['TOT_SALES'].count()
idx = pd.date_range('2018-12-01', '2019-01-30')
transactions_by_date = transactions_by_date.reindex(idx, fill_value=0)
fig = px.line(transactions_by_date,
              title="Total sales by date",
              labels={'value': 'Date', 'index':'Total Sales'},
              template='plotly_dark')
fig.show()

We can see that the increase in sales occurs in the lead-up to Christmas and that there are zero sales on Christmas day itself. This is due to shops being closed on Christmas day.
Now that we are satisfied that the data no longer has outliers, we can move on to
creating other features such as brand of chips or pack size from PROD_NAME. We will
start with pack size.

In [19]:
#### Pack size
def get_size(text):
    text = re.sub('[&/]', '', text)
    size = re.sub('\D', '', text)
    return(int(size))

transactionData['PACK_SIZE'] = transactionData.apply(lambda x: get_size(x['PROD_NAME']), axis="columns")
transactionData.sort_values('PACK_SIZE', ascending=False)

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE
139622,2018-12-12,70,70132,68251,4,Dorito Corn Chp Supreme 380g,2,13.0,380
32144,2019-05-18,40,40026,36227,14,Smiths Crnkle Chip Orgnl Big Bag 380g,1,5.9,380
67137,2018-11-30,223,223113,223590,4,Dorito Corn Chp Supreme 380g,2,13.0,380
249038,2018-10-13,142,142256,143109,14,Smiths Crnkle Chip Orgnl Big Bag 380g,2,11.8,380
206337,2019-01-02,47,47439,42948,14,Smiths Crnkle Chip Orgnl Big Bag 380g,2,11.8,380
...,...,...,...,...,...,...,...,...,...
189801,2018-09-18,67,67252,65334,38,Infuzions Mango Chutny Papadums 70g,2,4.8,70
50822,2019-03-31,213,213184,213111,38,Infuzions Mango Chutny Papadums 70g,2,4.8,70
154828,2018-11-10,55,55143,49325,38,Infuzions Mango Chutny Papadums 70g,2,4.8,70
109817,2018-12-23,152,152069,150667,38,Infuzions Mango Chutny Papadums 70g,2,4.8,70


The largest size is 380g and the smallest size is 70g - seems sensible!

Next, let's plot a histogram of PACK_SIZE since we know that it is a categorical 
variable and not a continuous variable even though it is numeric.

In [36]:
fig = px.histogram(transactionData['PACK_SIZE'],
              title="The distribution of pack sizes",
              template='plotly_dark',
              labels={"value":"Pack sizes"})
fig.show()

Pack sizes created look reasonable.
Now to create brands, we can use the first word in PROD_NAME to work out the brand name...

In [51]:
def get_brand(text):
    text = re.sub('[&/]', ' ', text)
    return(text.split()[0])

transactionData['BRANDS'] = transactionData.apply(lambda x: get_brand(x['PROD_NAME']), axis="columns")
print(np.sort(transactionData['BRANDS'].unique()))

['Burger' 'CCs' 'Cheetos' 'Cheezels' 'Cobs' 'Dorito' 'Doritos' 'French'
 'Grain' 'GrnWves' 'Infuzions' 'Infzns' 'Kettle' 'NCC' 'Natural'
 'Pringles' 'RRD' 'Red' 'Smith' 'Smiths' 'Snbts' 'Sunbites' 'Thins'
 'Tostitos' 'Twisties' 'Tyrrells' 'WW' 'Woolworths']


Some of the brand names look like they are of the same brands - such as RED and RRD, which are both Red Rock Deli chips. Let's combine these together.


In [54]:
transactionData.loc[(transactionData.BRANDS == 'WW'),'BRANDS'] = 'Woolworths'
transactionData.loc[(transactionData.BRANDS == 'RRD'),'BRANDS'] = 'Red'
transactionData.loc[(transactionData.BRANDS == 'Snbts'),'BRANDS'] = 'Smiths'
transactionData.loc[(transactionData.BRANDS == 'Infzns'),'BRANDS'] = 'Infuzions'
transactionData.loc[(transactionData.BRANDS == 'NCC'),'BRANDS'] = 'Natural'
transactionData.loc[(transactionData.BRANDS == 'GrnWves'),'BRANDS'] = 'Grain'
transactionData.loc[(transactionData.BRANDS == 'Smith'),'BRANDS'] = 'Smiths'
transactionData.loc[(transactionData.BRANDS == 'Dorito'),'BRANDS'] = 'Doritos'
#### Check again
print(np.sort(transactionData['BRANDS'].unique()))

['Burger' 'CCs' 'Cheetos' 'Cheezels' 'Cobs' 'Doritos' 'French' 'Grain'
 'Infuzions' 'Kettle' 'Natural' 'Pringles' 'Red' 'Smiths' 'Sunbites'
 'Thins' 'Tostitos' 'Twisties' 'Tyrrells' 'Woolworths']


Now that we are happy with the transaction dataset, let's have a look at the customer dataset.

In [55]:
#### Examining customer data
customerData.head()

,LYLTY_CARD_NBR,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,YOUNG SINGLES/COUPLES,Premium
1,1002,YOUNG SINGLES/COUPLES,Mainstream
2,1003,YOUNG FAMILIES,Budget
3,1004,OLDER SINGLES/COUPLES,Mainstream
4,1005,MIDAGE SINGLES/COUPLES,Mainstream


In [58]:
display(customerData.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72637 entries, 0 to 72636
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   LYLTY_CARD_NBR    72637 non-null  int64 
 1   LIFESTAGE         72637 non-null  object
 2   PREMIUM_CUSTOMER  72637 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


None

In [116]:
df = customerData.groupby('LIFESTAGE').count()
fig = px.pie(df, values='LYLTY_CARD_NBR', names=df.index, template="plotly_dark", hole=.3,
             title='Propotion of lifestage')
fig.show()

In [118]:
# Propotion of LIFESTAGES
df = customerData.groupby('PREMIUM_CUSTOMER').count()
fig = px.pie(df, values='LYLTY_CARD_NBR', names=df.index, template="plotly_dark", hole=.3,
             title='Propotion between types of customer')
fig.show()

In [75]:
count_lifestage = pd.pivot_table(customerData, values="LYLTY_CARD_NBR", index="LIFESTAGE", columns="PREMIUM_CUSTOMER", aggfunc="count")
fig = px.bar(count_lifestage, template='plotly_dark')
fig.show()